In [4]:
import pandas as pd
import gc

path = "../"

In [6]:
#Loading the Excel document
file_path = path + "Premier League Data Analysis.xlsx"
xls = pd.ExcelFile(file_path)

#Loading the sheets into DataFrames
match_data_df = pd.read_excel(xls, sheet_name="Match Data")
season_summary_df = pd.read_excel(xls, sheet_name="Partial Data")

#Converting the "Date" column to datetime in Match Data
match_data_df["Date"] = pd.to_datetime(match_data_df["Date"], format="%d/%m/%y", errors='coerce')

#Filtering out matches before 19/08/2000
date_threshold = pd.to_datetime("19/08/2000", format="%d/%m/%Y")
match_data_df = match_data_df[match_data_df["Date"] >= date_threshold]

#Function to calculate Shots/ShotsOnTarget ratio for each team during a season
def calculate_shots_ratio(season, team, match_data):
    
    #Filtering matches for the given season and team (home or away)
    season_matches = match_data[match_data['Season'] == season]
    home_matches = season_matches[season_matches['HomeTeam'] == team]
    away_matches = season_matches[season_matches['AwayTeam'] == team]

    #Calculating home and away stats
    home_shots = home_matches['HS'].sum()  #Home Shots
    home_shots_on_target = home_matches['HST'].sum()  #Home Shots on Target
    away_shots = away_matches['AS'].sum()  #Away Shots
    away_shots_on_target = away_matches['AST'].sum()  #Away Shots on Target

    #Total shots and shots on target
    total_shots = home_shots + away_shots
    total_shots_on_target = home_shots_on_target + away_shots_on_target

    #Calculating and returning the ratio (handle division by zero)
    return total_shots_on_target / total_shots if total_shots > 0 else 0

#Adding a new column for Shots/ShotsOnTarget in the season summary
season_summary_df['Shots/ShotsOnTarget'] = 0.0

#Looping through the season summary and calculate the ratio for each team
for index, row in season_summary_df.iterrows():
    team = row['Team']
    season = row['Season']

    #Calculating the shots ratio for this team and season
    shots_ratio = calculate_shots_ratio(season, team, match_data_df)
    
    #Updating the season summary DataFrame with the calculated ratio
    season_summary_df.at[index, 'Shots/ShotsOnTarget'] = shots_ratio

#Saving the updated Season Summary back to the Excel file
output_file_path = path + "Shots_Partial.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print(f"Updated Season Summary saved to {output_file_path}")
gc.collect()

Updated Season Summary saved to C:\Users\antoi\Documents\University\IGP\Excel Documents\Shots_Partial.xlsx


483

In [8]:
#Function to calculate average yellow cards per game for each team during a season
def calculate_yellow_cards_ratio(season, team, match_data):
    
    #Filtering matches for the given season and team (home or away)
    season_matches = match_data[match_data['Season'] == season]
    home_matches = season_matches[season_matches['HomeTeam'] == team]
    away_matches = season_matches[season_matches['AwayTeam'] == team]

    #Calculating total yellow cards for home and away games
    home_yellows = home_matches['HY'].sum()  #Home Yellow Cards
    away_yellows = away_matches['AY'].sum()  #Away Yellow Cards

    #Total matches played (home + away)
    total_matches = len(home_matches) + len(away_matches)

    #Calculating and return the average yellow cards per game (handle division by zero)
    return (home_yellows + away_yellows) / total_matches if total_matches > 0 else 0

#Adding a new column for Average Yellow Cards per Game in the season summary
season_summary_df['Avg_Yellow_Cards/Game'] = 0.0

#Looping through the season summary and calculate the average yellow cards for each team
for index, row in season_summary_df.iterrows():
    team = row['Team']
    season = row['Season']

    #Calculating the average yellow cards per game for this team and season
    avg_yellow_cards = calculate_yellow_cards_ratio(season, team, match_data_df)
    
    #Updating the season summary DataFrame with the calculated average
    season_summary_df.at[index, 'Avg_Yellow_Cards/Game'] = avg_yellow_cards

#Saving the updated Season Summary back to the Excel file
output_file_path = path + "Yellow_Partial.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print(f"Updated Season Summary with Avg_Yellow_Cards/Game saved to {output_file_path}")
gc.collect()

Updated Season Summary with Avg_Yellow_Cards/Game saved to C:\Users\antoi\Documents\University\IGP\Excel Documents\Yellow_Partial.xlsx


653

In [10]:
#Function to calculate average red cards per game for each team during a season
def calculate_red_cards_ratio(season, team, match_data):
   
    #Filtering matches for the given season and team (home or away)
    season_matches = match_data[match_data['Season'] == season]
    home_matches = season_matches[season_matches['HomeTeam'] == team]
    away_matches = season_matches[season_matches['AwayTeam'] == team]

    #Calculating total yellow cards for home and away games
    home_reds = home_matches['HR'].sum()  #Home Red Cards
    away_reds = away_matches['AR'].sum()  #Away Red Cards

    #Total matches played (home + away)
    total_matches = len(home_matches) + len(away_matches)

    #Calculating and return the average red cards per game (handle division by zero)
    return (home_reds + away_reds) / total_matches if total_matches > 0 else 0

#Adding a new column for Average Yellow Cards per Game in the season summary
season_summary_df['Avg_Red_Cards/Game'] = 0.0

#Looping through the season summary and calculate the average yellow cards for each team
for index, row in season_summary_df.iterrows():
    team = row['Team']
    season = row['Season']

    #Calculating the average red cards per game for this team and season
    avg_red_cards = calculate_red_cards_ratio(season, team, match_data_df)
    
    #Updating the season summary DataFrame with the calculated average
    season_summary_df.at[index, 'Avg_Red_Cards/Game'] = avg_red_cards

#Saving the updated Season Summary back to the Excel file
output_file_path = path + "Red_Partial.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print(f"Updated Season Summary with Avg_Red_Cards/Game saved to {output_file_path}")
gc.collect()

Updated Season Summary with Avg_Yellow_Cards/Game saved to C:\Users\antoi\Documents\University\IGP\Excel Documents\Red_Partial.xlsx


695

In [12]:
#Function to calculate percentage of wins based on conditions
def calculate_percentage(matches, condition_col, result_value):
    total_matches = len(matches)
    if total_matches == 0:
        return 0  # Avoid division by zero
    wins = len(matches[matches[condition_col] == result_value])
    return (wins / total_matches) * 100

#Adding new columns for the 6 stats in the season summary dataframe
season_summary_df['Home_Win_Leading_HT(%)'] = 0.0
season_summary_df['Away_Win_Leading_HT(%)'] = 0.0
season_summary_df['Home_Win_Draw_HT(%)'] = 0.0
season_summary_df['Away_Win_Draw_HT(%)'] = 0.0
season_summary_df['Home_Win_Losing_HT(%)'] = 0.0
season_summary_df['Away_Win_Losing_HT(%)'] = 0.0

#Looping through the season summary and calculate percentages for each team
for index, row in season_summary_df.iterrows():
    team = row['Team']
    season = row['Season']
    
    #Filtering match data for the current season
    season_matches = match_data_df[match_data_df['Season'] == season]
    
    #Condition 1: Team is home, winning at HT (HTR == 'H') and winning at FT (FTR == 'H')
    home_leading_ht = season_matches[(season_matches['HomeTeam'] == team) & (season_matches['HTR'] == 'H')]
    season_summary_df.at[index, 'Home_Win_Leading_HT(%)'] = calculate_percentage(home_leading_ht, 'FTR', 'H')
    
    #Condition 2: Team is away, winning at HT (HTR == 'A') and winning at FT (FTR == 'A')
    away_leading_ht = season_matches[(season_matches['AwayTeam'] == team) & (season_matches['HTR'] == 'A')]
    season_summary_df.at[index, 'Away_Win_Leading_HT(%)'] = calculate_percentage(away_leading_ht, 'FTR', 'A')
    
    #Condition 3: Team is home, draw at HT (HTR == 'D') and winning at FT (FTR == 'H')
    home_draw_ht = season_matches[(season_matches['HomeTeam'] == team) & (season_matches['HTR'] == 'D')]
    season_summary_df.at[index, 'Home_Win_Draw_HT(%)'] = calculate_percentage(home_draw_ht, 'FTR', 'H')
    
    #Condition 4: Team is away, draw at HT (HTR == 'D') and winning at FT (FTR == 'A')
    away_draw_ht = season_matches[(season_matches['AwayTeam'] == team) & (season_matches['HTR'] == 'D')]
    season_summary_df.at[index, 'Away_Win_Draw_HT(%)'] = calculate_percentage(away_draw_ht, 'FTR', 'A')
    
    #Condition 5: Team is home, losing at HT (HTR == 'A') and winning at FT (FTR == 'H')
    home_losing_ht = season_matches[(season_matches['HomeTeam'] == team) & (season_matches['HTR'] == 'A')]
    season_summary_df.at[index, 'Home_Win_Losing_HT(%)'] = calculate_percentage(home_losing_ht, 'FTR', 'H')
    
    #Condition 6: Team is away, losing at HT (HTR == 'H') and winning at FT (FTR == 'A')
    away_losing_ht = season_matches[(season_matches['AwayTeam'] == team) & (season_matches['HTR'] == 'H')]
    season_summary_df.at[index, 'Away_Win_Losing_HT(%)'] = calculate_percentage(away_losing_ht, 'FTR', 'A')

#Saving the updated Season Summary back to the Excel file
output_file_path = path + "Stats_Partial-HTR.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print(f"Updated Season Summary with new stats saved to {output_file_path}")

Updated Season Summary with new stats saved to C:\Users\antoi\Documents\University\IGP\Excel Documents\Stats_Partial-HTR.xlsx


In [16]:
team_abbreviation_dict = {
    'NFO': "Nott'm Forest",
    'CHE': 'Chelsea',
    'TOT': 'Tottenham',
    'CRY': 'Crystal Palace',
    'SWN': 'Swansea',
    'MCI': 'Man City',
    'EVE': 'Everton',
    'COV': 'Coventry',
    'NOR': 'Norwich',
    'SOU': 'Southampton',
    'LIV': 'Liverpool',
    'WHU': 'West Ham',
    'QPR': 'QPR',
    'LEI': 'Leicester',
    'AVL': 'Aston Villa',
    'IPS': 'Ipswich',
    'ARS': 'Arsenal',
    'BLB': 'Blackburn',
    'BOL': 'Bolton',
    'LEE': 'Leeds',
    'NEW': 'Newcastle',
    'SHW': 'Sheffield Weds',
    'WIM': 'Wimbledon',
    'BRC': 'Bristol',
    'MID': 'Middlesbrough',
    'DER': 'Derby',
    'SUN': 'Sunderland',
    'FUL': 'Fulham',
    'WBA': 'West Bromwich Albion',
    'POR': 'Portsmouth',
    'CHA': 'Charlton',
    'WIG': 'Wigan',
    'BIR': 'Birmingham',
    'BUR': 'Burnley',
    'HUL': 'Hull',
    'WOL': 'Wolves',
    'SWA': 'Swansea',
    'REA': 'Reading',
    'STO': 'Stoke',
    'MUN': 'Man United',
    'CAR': 'Cardiff',
    'WAT': 'Watford',
    'HUD': 'Huddersfield',
    'BRI': 'Brighton',
    'SHU': 'Sheffield United',
    'BOU': 'Bournemouth'
}

In [18]:
abbreviation_to_name = team_abbreviation_dict
full_names_df = pd.read_excel(file_path, sheet_name="Season Summary")
abbreviations_df = pd.read_excel(file_path, sheet_name="Manager Changes")

#Replacing abbreviations in the 'Abbreviation' column with full team names using map()
abbreviations_df['Team'] = abbreviations_df['Team'].map(abbreviation_to_name)

#Saving the updated DataFrame back to Excel
output_file_path = path + "updated_premier_league_data.xlsx"
abbreviations_df.to_excel(output_file_path, index=False)
print(f"Updated data saved to {output_file_path}")
print(abbreviations_df)

        Season Team                 Date             Manager Out  \
0    1992-1993  NaN  1993-08-05 00:00:00            Brian Clough   
1    1992-1993  NaN            15/2/1993         Ian Porterfield   
2    1992-1993  NaN            30/6/1993              David Webb   
3    1992-1993  NaN            19/6/1993  D.Livermore/R.Clemence   
4    1992-1993  NaN            17/5/1993           Steve Coppell   
..         ...  ...                  ...                     ...   
405  2024-2025  NaN           15/12/2024          Russell Martin   
406  2024-2025  NaN           22/12/2024             Simon Rusk*   
407  2024-2025  NaN  2025-08-01 00:00:00         Julen Lopetegui   
408  2024-2025  NaN  2025-09-01 00:00:00              Sean Dyche   
409  2024-2025  NaN  2025-10-01 00:00:00        Leighton Baines*   

           Manager In  Days in Charge  # manager changes so far Relegated Y/N  \
0         Frank Clark            6700                         1             Y   
1          David Webb

In [20]:
#Function to convert 'YYYY/YY' format to 'YYYY-YYYY'
def convert_season_format(season):
    parts = season.split('/')
    if len(parts) == 2:
        start_year = parts[0]
        end_year = str(int(parts[0]) + 1) if len(parts[1]) == 2 else parts[1]
        return f"{start_year}-{end_year}"
    return season

#Applying the conversion function to the column with the 'YYYY/YY' format
abbreviations_df['Season'] = abbreviations_df['Season'].apply(convert_season_format)

#Saving the updated DataFrame back to Excel (optional)
output_file_path = path +"updated_season_format.xlsx"
abbreviations_df.to_excel(output_file_path, index=False)

In [22]:
#Reading the Manager Changes sheet and Season Summary sheet
manager_changes_df = pd.read_excel(file_path, sheet_name="Manager Changes")
season_summary_df = pd.read_excel(file_path, sheet_name="Partial Data")

#Counting the number of manager changes per team per season
manager_changes_count = manager_changes_df.groupby(['Season', 'Team']).size().reset_index(name='Manager Changes')

#Merging the counts into the Season Summary sheet
season_summary_df = season_summary_df.merge(
    manager_changes_count, 
    how='left', 
    left_on=['Season', 'Team'], 
    right_on=['Season', 'Team']
)

#Replacing NaN values with 0 (for teams with no manager changes)
season_summary_df['Manager Changes'] = season_summary_df['Manager Changes'].fillna(0).astype(int)

#Checking resulting column and saving back to the excel document
print(season_summary_df['Manager Changes'])
output_file_path = path +"manager_changes_counts_Partial.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print("The Season Summary sheet has been successfully updated with manager change counts.")

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     2
8     0
9     0
10    2
11    0
12    0
13    2
14    0
15    0
16    2
17    0
18    1
19    1
Name: Manager Changes, dtype: int32
The Season Summary sheet has been successfully updated with manager change counts.


In [24]:
#Filtering for rows where "Manager Out" or "Manager In" contains '*', indicating caretaker managers
caretaker_changes_df = manager_changes_df[
    manager_changes_df['Manager Out'].str.contains(r'\*', na=False) 
]

#Summing up the "Days in Charge" for caretaker managers for each team and season
caretaker_days = caretaker_changes_df.groupby(['Season', 'Team'])['Days in Charge'].sum().reset_index(name='Caretaker Days')

#Merging the caretaker days into the Season Summary sheet
season_summary_df = season_summary_df.merge(
    caretaker_days, 
    how='left', 
    left_on=['Season', 'Team'], 
    right_on=['Season', 'Team']
)

#Replacing NaN values with 0 (for teams with no caretaker managers)
season_summary_df['Caretaker Days'] = season_summary_df['Caretaker Days'].fillna(0).astype(int)

#Saving the data into a new column in the "Season Summary" sheet
output_file_path = path +"caretaker_days_Partial.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print("The Season Summary sheet has been successfully updated with manager change counts.")

The Season Summary sheet has been successfully updated with manager change counts.


In [26]:
#Ranking teams based on Total_Points first
season_summary_df['Rank'] = season_summary_df.groupby('Season')['Total_Points'] \
    .rank(method='min', ascending=False)

#Saving the newly created "Rank" column
output_file_path = path +"ranksperseason.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print("The Season Summary sheet has been successfully updated with team ranks.")

The Season Summary sheet has been successfully updated with team ranks.


In [8]:
#Loading the sheets into dataframes
match_data_df = pd.read_excel(xls, sheet_name="Match Data")
season_summary_df = pd.read_excel(xls, sheet_name="Partial Data")

#Defining a function to determine the season from a match date
def get_season_from_date(date_str):
    
    #Converting the string date to a datetime object
    date = pd.to_datetime(date_str, format='%d/%m/%y')
    
    #Getting the year and month
    year = date.year
    month = date.month
    
    #If the match is played after July, it's part of the next season
    if month >= 7:
        season = f"{year}-{year+1}"
    else:
        season = f"{year-1}-{year}"
    
    return season

#Applying the function to create a "Season" column in the match_data_df
match_data_df['Season'] = match_data_df['Date'].apply(get_season_from_date)

#Initialising a dictionary to store total corners for each team in each season
team_corner_dict = {}

#Processing each row of the season summary
for index, row in season_summary_df.iterrows():
    team = row['Team']
    season = row['Season']
    
    #Filtering the match data for the relevant season and the team
    team_matches = match_data_df[(match_data_df['Season'] == season) & 
                                 ((match_data_df['HomeTeam'] == team) | (match_data_df['AwayTeam'] == team))]
    
    #Calculating the total corners for the team in that season
    total_corners = 0
    for _, match in team_matches.iterrows():
        if match['HomeTeam'] == team:
            total_corners += match['HC']  #Home Corners
        else:
            total_corners += match['AC']  #Away Corners
    
    #Storing the total corners for the team in that season
    team_corner_dict[(team, season)] = total_corners

#Adding the total corners as a new column in the season summary dataframe
season_summary_df['Total_Corners'] = season_summary_df.apply(lambda row: team_corner_dict.get((row['Team'], row['Season']), 0), axis=1)

#Saving the newly created "Total_Corners" column
output_file_path = path +"corners.xlsx"
season_summary_df.to_excel(output_file_path, index=False)
print("The Season Summary sheet has been successfully updated with the total corners per team.")

The Season Summary sheet has been successfully updated with the total corners per team.
